In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd /content/drive/My Drive/Colab Notebooks/zDeep/

ModuleNotFoundError: No module named 'google.colab'

In [8]:
from decoder import Decoder
from utils import *
import data

import torch
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import time

import matplotlib.pyplot as plt ###

decoder = Decoder().to(device)

lr = 1e-3

lat_vecs = torch.nn.Embedding(1, latent_size, max_norm=1)
torch.nn.init.normal_(lat_vecs.weight.data, 0.0, sigma / np.sqrt(latent_size))

loss_l1 = torch.nn.L1Loss(reduction="sum")
optimizer = torch.optim.Adam(
    [
        {
            "params": decoder.parameters(),
            "lr": lr,
        },
        {
            "params": lat_vecs.parameters(),
            "lr": lr,
        },
    ]
)

def get_mean_latent_vector_magnitude(latent_vectors):
    return torch.mean(torch.norm(latent_vectors.weight.data.detach(), dim=1)).item()
# # loss_l1 = torch.nn.L1Loss(reduction="sum")
# loss_l1 = torch.nn.MSELoss(reduction="sum")
# optimizer = torch.optim.Adam(decoder.parameters(), lr)

loss_log = []
l2_log = []
l3_log = []
lat_mag_log = []
time_log = []
epochs = []

im_log = []
def log_im(sdf, x, epoch):
    len = sidelen**2
    sdf = sdf[:len].cpu().detach().numpy()
    x = x[:len, :].cpu().numpy()
    sdf = np.append(x, sdf, axis=1)
    img = samples_to_img(sdf)

    _, _, (lnd2, lnd3), (lrd3, lrd4) = get_normalization_params(img)
    t2 = np.degrees(np.arctan2(lnd3, lnd2)/2)
    t3 = np.degrees(np.arctan2(lrd4, lrd3))
    mag = get_mean_latent_vector_magnitude(lat_vecs)

    im_log.append(img)
    epochs.append(epoch)
    l2_log.append(t2)
    l3_log.append(t3)
    lat_mag_log.append(mag)

dog = np.load("dog/dog512.npy")
sdf_data = torch.from_numpy(dog).float()#[torch.randperm(dog.shape[0])].float()
sdf_data = sdf_data.reshape(-1, 3)
x = sdf_data[:, 0:2]
sdf = sdf_data[:, 2].unsqueeze(1)
log_im(sdf, x, -1)

def clamp_distances(distances, clamp_value):
    return torch.clamp(distances, -clamp_value, clamp_value)

def weighted_l1_loss(predictions, targets, weights):
    loss = weights * torch.abs(predictions - targets)
    return loss.sum()

clamp_dist = 0.1
num_epochs = 51
def trainnet():
    sdf_data = torch.from_numpy(dog).float()#[torch.randperm(dog.shape[0])].float()
    sdf_data = sdf_data.reshape(-1, 3)
    print(sdf_data.shape)

    for epoch in tqdm(range(num_epochs)):

        decoder.train()
        optimizer.zero_grad()

        start = time.time()
        loss_total = 0

        sdf_data.requires_grad = False

        sdf_data = sdf_data.reshape(-1, 3).to(device)
        num_sdf_samples = sdf_data.shape[0]

        # g = torch.arctan2(l4, l3).float()
        # print(torch.rad2deg(g))
        g = torch.tensor([0]).to(device)
        x = sdf_data[:, 0:2]
        z = lat_vecs(torch.tensor([0])).repeat(x.shape[0], 1).to(device)

        sdf_gt = sdf_data[:, 2].unsqueeze(1)

        pred_sdf = decoder(g, x, z)

        clamped_pred = pred_sdf
        clamped_gt = sdf_gt

        # weights = torch.exp(-torch.abs(sdf_gt) / clamp_dist)
        # loss = weighted_l1_loss(pred_sdf, sdf_gt, weights)

        # clamped_pred = clamp_distances(pred_sdf, clamp_dist+0.1)
        # clamped_gt = clamp_distances(sdf_gt, clamp_dist)
        loss = loss_l1(clamped_pred, clamped_gt) #/ num_sdf_samples
        loss.backward()
        loss_total += loss.item()
        optimizer.step()
        # break
        # break
        if epoch % int(num_epochs / 10) == 0 and epoch > 1:
            log_im(pred_sdf[:sidelen**2], x, epoch)

        end = time.time()
        seconds_elapsed = end - start

        loss_log.append(loss_total)
        time_log.append(seconds_elapsed)


trainnet()

fig, axs = plt.subplots(nrows=1, ncols=len(im_log), figsize=(4*len(im_log), 3))
for ax, im, e in zip(axs.ravel(), im_log, epochs):
    ax.imshow(im, cmap="gray")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(e, fontsize=32)
    # _, _, (lnd2, lnd3), (lrd3, lrd4) = get_normalization_params(im)
    # print(l2, l3, e)
    # a = np.degrees(np.arctan2(lnd3, lnd2)/2)
    # a = np.degrees(np.arctan2(lrd4, lrd3))
    # ax.set_xlabel(r"%.2f" % a, fontsize=32)

axs[0].set_title("gt", fontsize=32)
plt.show()


def plot_logs():
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(14, 4))
    ax1.plot(loss_log)
    ax1.set_title("Loss")
    ax2.plot(epochs, lat_mag_log)
    ax2.set_title("Norm")
    ax3.plot(epochs[1:], l2_log[1:], label=r"$\theta_2$")
    ax3.plot(epochs[1:], l3_log[1:], c="orange", label=r"$\theta_3$")
    ax3.set_title("Orientation")

    _, _, (lnd2, lnd3), (lrd3, lrd4) = get_normalization_params(im_log[0])
    t2 = np.degrees(np.arctan2(lnd3, lnd2)/2)
    t3 = np.degrees(np.arctan2(lrd4, lrd3))
    ax3.axhline(t2, linestyle="--", label=r"gt $\theta_2$")
    ax3.axhline(t3, c="orange", linestyle="--", label=r"gt $\theta_3$")

plot_logs()
plt.legend(loc=4)


torch.Size([26384, 3])


  0%|          | 0/51 [00:00<?, ?it/s]

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 26384 and the array at index 1 has size 16384